# Set puzzle parameters and create AoC Session

In [ ]:
# set puzzle parameters
PUZZLE_DAY = 6
PUZZLE_YEAR = 2024

# import from local packages
from aoc_solver import AoCSession, AoCSolver, AoCTester
AoC_SESSION = AoCSession.from_file()

# Import additional packages

In [ ]:
# import from standard library packages
import math

# import from third-party packages
import multiprocess as mp
import polars
from polars import DataFrame

# import from local packages
from aoc_solver import AoCMap, Direction

# Create solver class and instance

In [ ]:
class StuckInALoop(Exception):
    pass

class Solver(AoCSolver):
    
    @staticmethod
    def build_map(data: DataFrame) -> AoCMap:
        initial_x = initial_y = 0
        for initial_x, col in enumerate(data.iter_columns()):
            search_results = data.select(polars.arg_where(col == '^'))
            if not search_results.is_empty():
                initial_y = search_results.item()
                break
        return AoCMap(df=data.clone(), x=initial_x, y=initial_y, heading=Direction.N)
    
    @staticmethod
    def walk_map(lab_map: AoCMap) -> AoCMap:
        paths_walked = set()
        guard_on_map = True
        while guard_on_map:
            starting_position = lab_map.position
            path_contents = lab_map.look()
            try:
                obstruction_location = path_contents.index('#')
            except ValueError:
                obstruction_location = math.inf
                guard_on_map = False
            steps = min(len(path_contents), obstruction_location)
            breadcrumbs = ['X'] * (steps + 1)
            lab_map.update(values=breadcrumbs, offset=0)
            lab_map.walk(steps)
            lab_map.rotate(90)
            ending_position = lab_map.position
            path_walked = (starting_position, ending_position)
            if path_walked in paths_walked:
                raise StuckInALoop(f'Path repeated: {starting_position} to {ending_position}!')
            paths_walked.add((starting_position, ending_position))
        return lab_map
    
    def solve_part1(self, data: DataFrame) -> int:
        lab_map = self.build_map(data)
        updated_map = self.walk_map(lab_map)
        return updated_map.value_counts['X']

    def solve_part2(self, data: DataFrame) -> int:
        lab_map = self.build_map(data)
        x_max, y_max = lab_map.shape
        iteration_ranges = ( (lab_map, x, range(y_max)) for x in range(x_max) )

        def iteration_function(lap_map, x, range_of_y_values):
            loops_caught = 0
            for y in range_of_y_values:
                updated_map = lap_map.clone()
                start_position = updated_map.position
                updated_map.position = (x,y)
                updated_map.update(values=['#'], offset=0)
                updated_map.position = start_position
                try:
                    self.walk_map(updated_map)
                except StuckInALoop:
                    loops_caught += 1
            return loops_caught

        with mp.Pool(4) as pool:
            return_value = sum(pool.starmap(iteration_function, iteration_ranges))

        return return_value

In [ ]:
solver = Solver(PUZZLE_YEAR, PUZZLE_DAY, AoC_SESSION)

# Build part 1 test case(s)

In [ ]:
puzzle_instructions = solver.puzzle_instructions

part1_test_input = solver.get_value_after('For example:').create_polars()
print(f'{part1_test_input=}\n')

part1_test_output = solver.get_value_after('the guard will visit ').as_int
print(f'{part1_test_output=}\n')

In [ ]:
part_1_tester = AoCTester()
part_1_tester.add_test_case(part1_test_input, part1_test_output)

In [ ]:
%%time
part_1_tester.run_tests(solver.solve_part1)

# Determine part 1 solution

In [ ]:
%%time
puzzle_input = solver.puzzle_input.create_polars()
part1_solution = solver.solve_part1(puzzle_input)
print(f'{part1_solution=}\n')

# Add part 1 solution to part 1 test cases

In [ ]:
part_1_tester.add_test_case(puzzle_input, part1_solution)

In [ ]:
%%time
part_1_tester.run_tests(solver.solve_part1)

# Build part 2 test case(s)

In [ ]:
solver.download_instructions(overwrite=True)

In [ ]:
part2_test_input = part1_test_input
print(f'{part2_test_input=}\n')

part2_test_output = solver.get_value_after('in this example, there are ').as_int
print(f'{part2_test_output=}\n')

In [ ]:
part_2_tester = AoCTester()
part_2_tester.add_test_case(part2_test_input, part2_test_output)

In [ ]:
%%time
part_2_tester.run_tests(solver.solve_part2)

# Determine part 2 solution

In [ ]:
%%time
part2_solution = solver.solve_part2(puzzle_input)
print(f'{part2_solution=}\n')